In [1]:
from mimas.alphapept.find_features import find_features
from mimas.alphapept.load_mzml_data import load_mzml_data
from mimas.alphapept.find_features import find_features
import pandas as pd
import os
from toolsets.search import quick_search_sorted

def map_features(feature_mix, std_list_mix, adducts):
    feature_mix.sort_values(by = 'mz', inplace=True, ascending=True)
    mix_matched = pd.DataFrame()
    for index, row in std_list_mix.iterrows():
        compound_matched = pd.DataFrame()

        for a in adducts:
            adduct_matched = quick_search_sorted(feature_mix, 'mz', row[a]-0.005, row[a]+0.005)
            if len(adduct_matched)>0:
                # adduct_matched.insert(0, 'reference_mz', row[a])
                adduct_matched.insert(1, 'reference_name', row['Name'])
                adduct_matched.insert(1, 'reference_mz', row[a])
                adduct_matched.insert(1,'reference_inchikey', row['inchikey'])
                adduct_matched.insert(2, 'reference_adduct', a)

                # adduct_matched.insert(3, 'reference_rt', row['reference_rt'])
                adduct_matched.insert(4, 'reference_smiles', row['smiles'])
                adduct_matched.insert(6, 'reference_mix', row['Mix'])
                adduct_matched.insert(7, 'reference_rt', row['RT'])
                compound_matched  = pd.concat([compound_matched, adduct_matched], ignore_index=True)
            # return compound_matched
        if row['RT']==row['RT'] and len(compound_matched)>0:
            rt_offsets = abs(compound_matched['rt_apex']-row['RT'])
            compound_matched['rt_offset']=rt_offsets
            compound_matched = compound_matched[compound_matched['rt_offset']<5/60]
            compound_matched.drop(columns=['rt_offset'], inplace =True)
        elif len(compound_matched)>0:
            compound_matched.sort_values(by = 'ms1_int_max_apex', inplace=True, ascending=False)
            compound_matched.drop_duplicates(subset=['reference_adduct'], keep = 'first', inplace=True)

            # compound_matched = pd.concat([compound_matched, adduct_matched], ignore_index=True)
        mix_matched = pd.concat([mix_matched, compound_matched], ignore_index=True)
    return(mix_matched)

# Setup workingspace

In [ ]:
master_dir = 'some_dir' # only line need to edit

In [ ]:

pl_dir = os.path.join(master_dir, 'pl')
mzml_dir = os.path.join(master_dir, 'mzml')
matched_dir = os.path.join(master_dir, 'matched')
for f in [pl_dir, mzml_dir, matched_dir]:
    if os.path.exists(f)==False:
        os.makedirs(f)

# extract features

In [ ]:
def get_file_list(dir, tail, with_tail = False):
    file_list = []
    for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith(tail):
                if with_tail == True:
                    file_list.append(file)
                else:
                    file_list.append(file.split('.')[0])
    return(file_list)

In [2]:
mzml_list = (mzml_dir, '.mzML')
for f in mzml_list:
    ms_file_data = load_mzml_data(os.path.join(mzml_dir, f+'.mzML'))
    feature_temp =find_features(ms_file_data)
    feature_temp.to_csv(os.path.join(pl_dir, f+'.csv'), index = False)
#use for loop to extract all features

/Users/fanzhoukong/Documents/GitHub/GT_src/mimas/alphapept/find_features.py:284: RuntimeWarning: divide by zero encountered in divide
  feature_sn = feature_apex_xic / feature_noise * 2
/Users/fanzhoukong/Documents/GitHub/GT_src/mimas/alphapept/find_features.py:284: RuntimeWarning: invalid value encountered in divide
  feature_sn = feature_apex_xic / feature_noise * 2


# map features

In [5]:
std_list = pd.read_csv(os.path.join(master_dir, 'enlongated_list.csv'))


In [8]:
mapped_all = pd.DataFrame()
for mix in std_list['Mix'].unique():
    feature_mix = pd.read_csv(os.path.join(pl_dir, mix+'.csv'))
    std_list_mix = std_list[std_list['Mix']==mix]
    mapped_features = map_features(feature_temp, std_list_mix, adducts=['[M+H]+'])# extedn adducts to all adducts you want
    mapped_all = pd.concat([mapped_all, mapped_features], ignore_index=True)

In [9]:
mapped_all

,mz,reference_inchikey,reference_adduct,reference_mz,reference_smiles,reference_name,reference_mix,reference_rt,mz_std,mz_most_abundant,...,rt_end_idx,sn_ratio,rt_length,rt_right,rt_left,rt_tail,rt,feature_idx,ms2_idx,msms
0,253.174856,FCUGGFFHQXNXJN-MNRPPXDRSA-N,[M+H]+,253.1798,C[C@@]\1(CC[C@@]2([C@H](CC2(C)C)/C(=C\C=C1)/CO...,Punctaporonin B,1,0,0.000376,253.174856,...,50.0,NaN,0.038765,0.007753,0.031012,0.250000,0.077796,1757,NaN,NaN
1,170.150286,HDVAWXXJVMJBAR-VHSXEESVSA-N,[M+H]+,170.1539,C1CCN2CCC[C@H]([C@H]2C1)CO,Lupinine,1,0,0.000658,170.150286,...,19.0,NaN,0.007396,0.002465,0.004931,0.500000,0.044186,1515,NaN,NaN
2,170.150581,HDVAWXXJVMJBAR-VHSXEESVSA-N,[M+H]+,170.1539,C1CCN2CCC[C@H]([C@H]2C1)CO,Lupinine,1,0,0.000222,170.150581,...,40.0,NaN,0.026116,0.020767,0.005349,3.882353,0.077749,2576,NaN,NaN
3,170.151767,HDVAWXXJVMJBAR-VHSXEESVSA-N,[M+H]+,170.1539,C1CCN2CCC[C@H]([C@H]2C1)CO,Lupinine,1,0,0.000304,170.151767,...,11.0,NaN,0.017370,0.004963,0.012407,0.400000,0.024254,469,NaN,NaN
4,170.152457,HDVAWXXJVMJBAR-VHSXEESVSA-N,[M+H]+,170.1539,C1CCN2CCC[C@H]([C@H]2C1)CO,Lupinine,1,0,0.000581,170.152457,...,27.0,NaN,0.005152,0.000000,0.005152,0.000000,0.069800,2380,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,373.094585,DSLZVSRJTYRBFB-DUHBMQHGSA-N,[M+H]+,373.0981,[C@@H]([C@@H]([C@H](C(=O)O)O)O)([C@@H](C(=O)O)O)O,MUCIC ACID_glycoside,15,0,0.000385,373.094585,...,17.0,NaN,0.010050,0.010050,0.000000,inf,0.034150,1324,NaN,NaN
162,138.088637,DZGWFCGJZKJUFP-UHFFFAOYSA-N,[M+H]+,138.0913,C1=CC(=CC=C1CCN)O,TYRAMINE,15,0,0.000075,138.088637,...,75.0,NaN,0.178509,0.166475,0.012034,13.833333,0.036492,1011,NaN,NaN
163,355.100147,CWVRJTMFETXNAD-JUHZACGLSA-N,[M+H]+,355.1024,C1[C@H]([C@H]([C@@H](C[C@@]1(C(=O)O)O)OC(=O)/C...,Chlorogenic Acid,15,0,0.001623,355.100147,...,15.0,NaN,0.002602,0.000000,0.002602,0.000000,0.039200,1369,NaN,NaN
164,355.100495,CWVRJTMFETXNAD-JUHZACGLSA-N,[M+H]+,355.1024,C1[C@H]([C@H]([C@@H](C[C@@]1(C(=O)O)O)OC(=O)/C...,Chlorogenic Acid,15,0,0.000127,355.100495,...,2.0,NaN,0.002483,0.002483,0.000000,inf,0.004400,110,NaN,NaN
